# Machine learning mini-project 2
### Importing the libraries and loading the dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import ssl
import string

# Connection function
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words("english")

In [ ]:
df = pd.read_table("Sentiment140.tenPercent.sample.tweets.tsv")
df.head()

## Pre-processing the data

In [ ]:
# Convert the text to lowercase
df['tweet_text'] = df['tweet_text'].apply(lambda x: x.lower())

In [ ]:
# Remove punctuations
def remove_punctuations(text):
    translator = text.maketrans('', '', string.punctuation)
    return text.translate(translator)

df["tweet_text"] = df["tweet_text"].apply(remove_punctuations)

In [ ]:
# Remove stopwords
df["tweet_text"] = df["tweet_text"].replace(stop, "")

In [ ]:
# Create a new dataframe
df_2 = df.copy()

In [ ]:
df.shape

## Logistic Regression, Machine Learning method 1


In [ ]:
# Splitting into test and training data for method 1
X_train, X_test, y_train, y_test = train_test_split(df['tweet_text'], df['sentiment_label'], test_size=0.2, random_state=42)

In [ ]:
# Vectorizing the data for method 1
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Tokenize the data to turn it into a list of words
df["tweet_text"] = df["tweet_text"].apply(word_tokenize)

In [ ]:
#Stemming data for both methods

stemmer = PorterStemmer()
def apply_stemmer(text):
    return [stemmer.stem(word) for word in text]

df['tweet_text'] = df['tweet_text'].apply(apply_stemmer)

In [ ]:
lr = LogisticRegression(n_jobs=-1, max_iter=10000)
lr.fit(X_train_vec, y_train)
y_pred_lr = lr.predict(X_test_vec)

In [ ]:
# Print the accuracy of the model
print("Accuracy:", accuracy_score(y_test, y_pred_lr))

In [ ]:
# Print the confusion matrix of the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

In [ ]:
# Print the classification report of the model
print("Classification Report:\n", classification_report(y_test, y_pred_lr))

## Random Forest Classifier, Mahine learning method 2

In [ ]:
# Splitting into test and training data for method 1
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_2['tweet_text'], df['sentiment_label'], test_size=0.2, random_state=42)

In [ ]:
# Vectorizing the data for method 1
vectorizer_2 = TfidfVectorizer()
X_train_rf = vectorizer_2.fit_transform(X_train_2)
X_test_rf = vectorizer_2.transform(X_test_2)

In [ ]:
# Scaling data to work better with the method.
sc = StandardScaler(with_mean=False)
X_train_rf = sc.fit_transform(X_train_rf)
X_test_rf = sc.transform(X_test_rf)

In [ ]:
# Tokenize the data to turn it into a list of words
df_2["tweet_text"] = df_2["tweet_text"].apply(word_tokenize)

In [ ]:
#Stemming data for method 2
df_2['tweet_text'] = df_2['tweet_text'].apply(apply_stemmer)

In [ ]:
# 
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_rf, y_train)
y_pred_rf = rf_model.predict(X_test_rf)

In [ ]:
# Print the accuracy of the model
print('Accuracy of Random Forest Model:', accuracy_score(y_test_2, y_pred_rf))

In [ ]:
# Print the confusion matrix of the model
print('Confusion Matrix:\n', confusion_matrix(y_test_2, y_pred_rf))

In [ ]:
# Print the classification report of the model
print('Classification Report:\n', classification_report(y_test_2, y_pred_rf))

In [ ]:
# Calculate evaluation metrics for Random Forest Classifier
rf_accuracy = accuracy_score(y_test_2, y_pred_rf)
rf_precision = precision_score(y_test_2, y_pred_rf, average='weighted')
rf_recall = recall_score(y_test_2, y_pred_rf, average='weighted')
rf_f1 = f1_score(y_test_2, y_pred_rf, average='weighted')

In [ ]:
# Calculate evaluation metrics for Logistic Regression
lr_accuracy = accuracy_score(y_test_2, y_pred_lr)
lr_precision = precision_score(y_test_2, y_pred_lr, average='weighted')
lr_recall = recall_score(y_test_2, y_pred_lr, average='weighted')
lr_f1 = f1_score(y_test_2, y_pred_lr, average='weighted')

## Visualizing and comparing the results

In [ ]:
# Create a bar chart to compare the evaluation metrics
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
rf_scores = [rf_accuracy, rf_precision, rf_recall, rf_f1]
lr_scores = [lr_accuracy, lr_precision, lr_recall, lr_f1]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, rf_scores, width, label='Random Forest')
rects2 = ax.bar(x + width/2, lr_scores, width, label='Logistic Regression')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Score')
ax.set_title('Comparison of Methods')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(loc="center right")

plt.show()

In [ ]:
# Create confusion matrices for each model
rfc_cm = confusion_matrix(y_test, y_pred_rf)
lr_cm = confusion_matrix(y_test_2, y_pred_lr)

# Plot the confusion matrices
plt.figure(figsize=(8,6))
sns.heatmap(rfc_cm, annot=True, cmap='Greens', fmt='g', cbar=False)
plt.title('Random Forest Classifier Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

plt.figure(figsize=(8,6))
sns.heatmap(lr_cm, annot=True, cmap='Blues', fmt='g', cbar=False)
plt.title('Logistic Regression Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()